In [156]:
import nltk
import pandas as pd
import re

In [157]:
text = pd.read_csv('Data/dataset.csv')

In [158]:
text.head()

,movie_name,script
0,12 Monkeys,Twelve MonkeysTWELVE MONKEYS An o...
1,2001 A Space Odyssey,2001: A SPACE ODYSSEY Screenplay ...
2,2012,...
3,28 Days Later,28 DAYS LATER ...
4,9,...


In [163]:
text.dropna(inplace=True)

In [164]:
text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165 entries, 0 to 167
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   movie_name  165 non-null    object
 1   script      165 non-null    object
dtypes: object(2)
memory usage: 3.9+ KB


In [142]:
# # Removing punctuation
# text['script'] = text['script'].map(lambda x: re.sub('[,\.!?]', ' ', x))

# pre-processing

In [165]:
import nltk
from nltk.corpus import stopwords

In [166]:
stop_words = stopwords.words('english')
# stop_words.extend([])

In [167]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [168]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/anna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [169]:
def tokenize_list(texts):
    word_list = []
    for item in range(len(texts)):
        
        word_list.append(nltk.word_tokenize(texts[item]))
    return word_list

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

def lemmatize(text):
    for item in range(len(text)):
        text[item]=  [lemmatizer.lemmatize(w,pos='v') for w in text[item]]
        
    return text

In [170]:
text_list = tokenize_list(text.script.tolist())
text_list = remove_stopwords(text_list)
text_list = lemmatize(text_list)

In [171]:
len(text_list)

165

In [173]:
import gensim.corpora as corpora

# Create dictionary
id2word = corpora.Dictionary(text_list)


# Create corpus: Word frequency 
corpus = [id2word.doc2bow(text) for text in text_list]

# View
print(corpus[:1][0][:30])

[(0, 3), (1, 1), (2, 1), (3, 4), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 4), (14, 2), (15, 2), (16, 1), (17, 1), (18, 1), (19, 2), (20, 3), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 15)]


In [181]:
from tqdm import tqdm
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel

In [182]:
# Função para determinar a melhor quantidade de tópicos para a modelagem
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=5):
    coherence_values = []
    model_list = []
    for num_topics in tqdm(range(start, limit, step)):
        model = LdaMulticore(corpus, id2word=id2word, num_topics=10)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values


# Colocando parametros na função
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=text_list, start=2, limit=30, step=2)
# Mostrando visualmente a quantidade de tópicos
limit=30; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Tópicos")
plt.ylabel("Score de Coerência")
plt.legend(("Valores de Coerência"), loc='best')
plt.show()
# Lista dos valores de coerência, para melhor identificar o ponto de inflexão do gráfico
for m, cv in zip(x, coherence_values):
    print("A quantidade de tópicos =", m, " tem um valor de coerência de ", round(cv, 4))

  0%|          | 0/14 [40:25<?, ?it/s]Process ForkPoolWorker-10:
Process ForkPoolWorker-9:
Process ForkPoolWorker-14:
Process ForkPoolWorker-13:
Process ForkPoolWorker-11:
Process ForkPoolWorker-12:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):



In [ ]:
#https://medium.com/somos-tera/como-modelar-t%C3%B3picos-atrav%C3%A9s-de-latent-dirichlet-allocation-lda-atrav%C3%A9s-da-biblioteca-gensim-1fa17357ad4b